## WideNet prototype

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30447 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 4)
[+] Number of entries in each category:
[+] Training:
 [ 7697. 10911. 12259.  1133.]
[+] Validation:
 [1947. 2713. 3040.  300.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Feed the input vector into the embedding layer, then feed the resulting sequence into a bidirectional LSTM layer.

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(4,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(embedded_sequences)

In [4]:
inception = [] # append our layers in parallel

# kernel 7: best accuracy, some over fitting!
l_conv_7 = Conv1D(filters=10,kernel_size=7,activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_lstm1)
# kernel 2: good accuracy
l_conv_2 = Conv1D(filters=10,kernel_size=2,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 6: good accuracy
l_conv_6 = Conv1D(filters=10,kernel_size=6,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 3: good accuracy, no over fitting
l_conv_3 = Conv1D(filters=10,kernel_size=3,activation='relu')(l_lstm1)

inception.append(l_conv_7)
inception.append(l_conv_2)
inception.append(l_conv_6)
inception.append(l_conv_3)

# poorer performing layers
l_conv_4 = Conv1D(filters=10,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_4 = Dropout(0.4)(l_conv_4)
l_conv_5 = Conv1D(filters=10,kernel_size=5,activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_lstm1)
l_drop_5 = Dropout(0.4)(l_conv_5)
l_conv_8 = Conv1D(filters=10,kernel_size=8,activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_lstm1)
l_drop_8 = Dropout(0.4)(l_conv_8)

inception.append(l_drop_4)
inception.append(l_drop_5)
inception.append(l_drop_8)

In [5]:
# last parallel layer in inception: max-pooling layer
l_pool_i1 = MaxPooling1D(3)(l_lstm1)
l_conv_1 = Conv1D(filters=10,kernel_size=1,
                    activation='relu',kernel_regularizer=regularizers.l2(0.02))(l_pool_i1)
inception.append(l_conv_1)

In [6]:
l_merge = Concatenate(axis=1)(inception)
l_pool = MaxPooling1D(4)(l_merge)
l_drop = Dropout(0.3)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

In [7]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [8]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [9]:
model.summary()
model.save('WideNet.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      6089600     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 8)        6560        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 27, 10)       330         bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_6 (

In [ ]:
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 22s 691us/step - loss: 1.1645 - acc: 0.4879 - lr: 6.0000 - val_loss: 1.0546 - val_acc: 0.5636 - val_lr: 6.0000
Epoch 2/200
32000/32000 [==============================] - 21s 669us/step - loss: 1.0235 - acc: 0.5697 - lr: 1.0000 - val_loss: 1.0197 - val_acc: 0.5735 - val_lr: 1.0000
Epoch 3/200
32000/32000 [==============================] - 22s 685us/step - loss: 1.0084 - acc: 0.5754 - lr: 1.0000 - val_loss: 1.0136 - val_acc: 0.5745 - val_lr: 1.0000
Epoch 4/200
32000/32000 [==============================] - 23s 720us/step - loss: 1.0040 - acc: 0.5767 - lr: 1.0000 - val_loss: 1.0155 - val_acc: 0.5729 - val_lr: 1.0000
Epoch 5/200
32000/32000 [==============================] - 22s 698us/step - loss: 0.9986 - acc: 0.5807 - lr: 1.0000 - val_loss: 1.0110 - val_acc: 0.5754 - val_lr: 1.0000
Epoch 6/200
32000/32000 [==============================] - 23s 726us/step - loss: 

In [ ]:
import pandas as pd
model.save('WideNet.h5')
pd.DataFrame(model_log.history).to_csv("history-inception.csv")